In [ ]:
!pip install svgpathtools shapely svgwrite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import math
from svgpathtools import svg2paths, wsvg
from shapely.geometry import LineString

def simplify_svg(input_file, output_file, tolerance_mm=1.0, scale=1.0):
    """
    Упрощает SVG с точностью в миллиметрах.
    tolerance_mm — насколько можно отклоняться от оригинальной линии.
    scale — множитель перевода единиц SVG в мм (если нужно).
    """
    paths, attributes = svg2paths(input_file)
    simplified_paths = []

    for path in paths:
        points = []
        for seg in path:
            points.append((seg.start.real * scale, seg.start.imag * scale))
        if len(points) < 2:
            continue

        line = LineString(points)
        simplified = line.simplify(tolerance=tolerance_mm)
        simplified_paths.append(simplified)

    # сохраняем в новый SVG
    from svgwrite import Drawing
    dwg = Drawing(output_file, profile='tiny')
    for s in simplified_paths:
        if s.is_empty:
            continue
        d = "M " + " L ".join(f"{x:.2f},{y:.2f}" for x, y in s.coords)
        dwg.add(dwg.path(d=d, fill="none", stroke="black", stroke_width=0.2))
    dwg.save()

# пример использования:
simplify_svg("ladyghoststl-2-014.svg", "/simplified/simplified_01mm.svg", tolerance_mm=0.1)
# simplify_svg("input.svg", "simplified_5mm.svg", tolerance_mm=5.0)

Можно задать tolerance_mm=1.0 или 5.0 — это будет буквально «точность упрощения».
Поддерживает как миллиметры, так и другие единицы, если ты задашь scale